In [5]:
print("this is a test" + chr(0) + "string")

this is a test string


In [13]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
print(type(utf8_encoded))
list(utf8_encoded)

print(len(test_string))
print(len(utf8_encoded))

print(utf8_encoded.decode("utf-8"))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
<class 'bytes'>
13
23
hello! こんにちは!


In [19]:
import regex as re


PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [ ]:
for match in re.finditer(PAT, "some text that i'll pre-tokenize"):
    print(f'"{match.group()}"')


"some"
" text"
" that"
" i"
"'ll"
" pre"
"-"
"tokenize"
